Modeling the brusselator

In [3]:
import numpy as np
from scipy.fft import fftn, ifftn
%matplotlib inline

Brusselator:

$$\dot u = \triangle u +a - (b+1)u + u^2v$$
$$ \dot v = \triangle v + bu - u^2v$$
or in other words

$$\dot u = \triangle u f_u$$
$$ \dot v = \triangle v +f_v$$
This is in 2D. With random initial data. Do I need to worry about initial conditions? No!
I also want to be able to add a new function and a parameter

In [4]:
N = 64
dt = 0.005
fu = lambda a,b,u,v: a + (u**2)*v - (b+1)*u
fv = lambda a,b,u,v: b*u-(u**2)*v
L = 10*2*np.pi
NUM_STEPS = 1000
a = 1
b = 3
D0 = 1 
D1 = 0.1
# put only things I want to change
# don't want to change diffusion too much. At least for now.



In [5]:
def create_initial_array(num_of_nodes = N, ampl = 0.1):
    #return initial concentration of substance. Can be random, should be positive
    return np.random.rand(num_of_nodes,num_of_nodes) * ampl
# for now
def create_wavenumber_array(num_modes=N, L = L):
    # create array for wavenumbers in scipy format.
    num_modes = num_modes - num_modes%2 # make num of nodes even. Why?
    wavenum_array = np.zeros(num_modes)
    wavenum = 2*np.pi/L
    p1 = np.array([(wavenum * i) for i in range(int(num_modes/2))])
    
    p2 = np.array([wavenum * (-num_modes+n) for n in range(int(num_modes/2), num_modes)])
    wavenum_array = np.concatenate((p1,p2))
    return wavenum_array
def create_time_operator(wavenums, diffusion_coeff, timestep = dt):
    ## create a time evolution operator
    num_nodes = wavenums.size
    operator = np.empty((num_nodes,num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            wavenumber = -(wavenums[i]**2+wavenums[j]**2)
            operator[i,j] = np.exp(timestep*diffusion_coeff*wavenumber)
    return operator

In [6]:
def perform_simulation(a=a,b=b, timestep = dt, fu = fu, fv = fv, N = N, NUM_STEPS=NUM_STEPS):
    
    # initialize data
    initial_data_u = create_initial_array(N)
    initial_data_v = create_initial_array(N)
    wavenums = create_wavenumber_array(N, L)

    ## initialize operators
    operator_u = create_time_operator(wavenums, D0, timestep)
    operator_v = create_time_operator(wavenums, D1, timestep)
    
    ##initialize returned data
    time = np.empty(NUM_STEPS)
    time[0] = 0
    shape = time.shape+initial_data_u.shape
    conc_u = np.empty(shape)
    conc_v = np.empty(shape)
    conc_u[0] = initial_data_u
    conc_v[0] = initial_data_v

    ## begin calculations
    for i in range(NUM_STEPS-1):
        # initialize

        u = conc_u[i]
        v = conc_v[i]
        ## apply euler scheme
        nonlin_u = u + fu(a,b,u,v)*dt
        nonlin_v = v + fv(a,b,u,v)*dt
        ## use Fourier transform
        ## NOTES: I could combine u and v into complex array.
        fft_u = fftn(nonlin_u)
        fft_v = fftn(nonlin_v)

        ## perform timestep in fourier domain
        fft_u = fft_u * operator_u
        fft_v = fft_v * operator_v

        ## go back
        u = ifftn(fft_u).real
        v = ifftn(fft_v).real

        ## record the data
        time[i+1] = time[i] + timestep
        conc_u[i+1] = u
        conc_v[i+1] = v

    return time, conc_u, conc_v


In [7]:
def perform__trunc_simulation(a=a,b=b, timestep = dt, fu = fu, fv = fv, N = N, NUM_STEPS=NUM_STEPS, savetimes = [0, N//2, N-2]):
    
    # initialize data
    initial_data_u = create_initial_array(N)
    initial_data_v = create_initial_array(N)
    wavenums = create_wavenumber_array(N, L)

    ## initialize operators
    operator_u = create_time_operator(wavenums, D0, timestep)
    operator_v = create_time_operator(wavenums, D1, timestep)
    
    ##initialize data
    u = initial_data_u
    v = initial_data_v

    ## initialize saved data
    # shape = savetimes.shape+initial_data_u.shape
    u_saved = []
    v_saved = []
    ## begin calculations
    for i in range(NUM_STEPS-1):
        # initialize saving date
        if i in savetimes:
            u_saved.append(u)
            v_saved.append(v)
        ## apply euler scheme
        nonlin_u = u + fu(a,b,u,v)*dt
        nonlin_v = v + fv(a,b,u,v)*dt
        ## use Fourier transform
        ## NOTES: I could combine u and v into complex array.
        fft_u = fftn(nonlin_u)
        fft_v = fftn(nonlin_v)

        ## perform timestep in fourier domain
        fft_u = fft_u * operator_u
        fft_v = fft_v * operator_v

        ## go back
        u = ifftn(fft_u).real
        v = ifftn(fft_v).real

        ## record the data

    return u_saved, v_saved, savetimes


In [8]:
perform_simulation(NUM_STEPS= 10)

(array([0.   , 0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 ,
        0.045]),
 array([[[3.46196164e-02, 7.87705968e-02, 9.96736789e-02, ...,
          8.96793388e-02, 8.02114982e-02, 8.22918093e-02],
         [9.29579622e-02, 4.61418198e-02, 6.94167905e-02, ...,
          5.43602858e-02, 9.02559748e-05, 5.49055873e-02],
         [6.44073725e-02, 4.44753859e-02, 8.75177224e-02, ...,
          2.87157971e-02, 4.06037366e-02, 4.80139000e-02],
         ...,
         [3.92749472e-02, 7.45173140e-02, 4.28613912e-02, ...,
          9.23513919e-02, 8.97310054e-02, 7.26504850e-02],
         [6.76885893e-06, 3.32393062e-02, 2.04712778e-02, ...,
          4.03826109e-02, 6.48088136e-02, 9.41398949e-02],
         [5.51716604e-02, 9.10194450e-02, 7.55306908e-03, ...,
          9.35028672e-02, 2.86033548e-02, 9.22156614e-02]],
 
        [[4.04160388e-02, 8.19388099e-02, 1.01173541e-01, ...,
          9.20812803e-02, 8.27542426e-02, 8.49903341e-02],
         [9.46050882e-02, 5.11832493e-02,

In [9]:
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
# Plot u(x,y) for different time points (2D)

dt = 0.05
data_u, data_v, savetimes = perform__trunc_simulation(NUM_STEPS= 10**3)
xs = np.arange(0.0,L,L/N)
ys = np.arange(0.0,L,L/N)

X,Y = np.meshgrid(xs,ys)

from matplotlib import animation, rc
from IPython.display import HTML

fig = plt.figure(figsize=(9.0,3.5))
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(232)
ax3 = fig.add_subplot(233)

ax1.pcolormesh(X, Y, data_u[savetimes[0]], shading='auto')
ax2.pcolormesh(X, Y, data_u[savetimes[1]], shading='auto')
ax3.pcolormesh(X, Y, data_u[savetimes[2]], shading='auto')
ax4 = fig.add_subplot(234)
ax5 = fig.add_subplot(235)
ax6 = fig.add_subplot(236)

ax4.pcolormesh(X, Y, data_v[savetimes[0]], shading='auto')
ax5.pcolormesh(X, Y, data_v[savetimes[1]], shading='auto')
ax6.pcolormesh(X, Y, data_v[savetimes[2]], shading='auto')

plt.show()

AttributeError: 'list' object has no attribute 'shape'

What do I want from the simulation?
I want to be able to change the parameters rather easily. Or display a bunch of plots 
What parameters do I have?

def plot_simulation(a,b):
